# Nations and Conflict

In [1]:
import pandas as pd
import numpy as np
import pickle

import altair as alt


# importing local libraries for drawings (because they clutter)
# in Github, you can search  
#   They should still be available within the git repo but it's pretty much template stuff from altair
import sys
sys.path.append('../')
from lib.builders import locators,towns,nations
import lib.drawMaps as draw

with (open('world_2.p', 'rb')) as pickle_file:
    world = pickle.load(pickle_file)
    
world

Towns are python objects, each with thier own diplomacy. Land spaces are attributed to nations in a previous stage. 

In [2]:
world.towns

[capitol of Spinmoral: population: 4 location: [22,39] founded 0,
 capitol of Splitcoil: population: 5 location: [22,14] founded 1,
 capitol of Snowgarden: population: 3 location: [34,25] founded 2,
 town of Godridge: population: 4 location: [18,18] founded 2,
 capitol of Leafcrest: population: 6 location: [2,30] founded 3,
 town of Warcrawl: population: 4 location: [37,4] founded 3,
 capitol of Badgerhaven: population: 5 location: [37,8] founded 3,
 capitol of Earthcrest: population: 2 location: [3,10] founded 4,
 capitol of Faircrawl: population: 3 location: [16,0] founded 4,
 town of Heartplace: population: 3 location: [20,2] founded 4,
 town of Lilyfellow: population: 2 location: [25,8] founded 5,
 town of Falljaw: population: 2 location: [20,13] founded 5,
 town of Flameward: population: 4 location: [9,27] founded 5,
 capitol of Oakford: population: 3 location: [35,32] founded 5,
 town of Founder'shail: population: 3 location: [31,32] founded 6,
 town of Lionspring: population: 3 

In [3]:
town = world.towns[1]
print(f"{town.name} is loyal to the nation of {town.nation}")
print(f"{town.diplomacy}")

Splitcoil is loyal to the nation of Splitcoil
{'nation': 'Oakridge', 'national fealty': 1}


In [4]:
#nations contain cities
nameparts = pd.read_csv('namegen.csv')

class person:
    def __init__(self,role,location):
        self.name = self.personNameGenerator()
        self.role = role
        self.nation = location
        self.location = location
        self.loyalty = .5
        #temperment determines the 'viciousness' of a person's decisons
        self.temperment = np.random.random_sample()
        
        self.setLoyaltytoCrown()
        
    def __repr__(self):
        return f"{self.name}, {self.role}, residing in {self.nation}"
        
    def personNameGenerator(self):
        return (np.random.choice(nameparts['1']) + \
                np.random.choice(nameparts['2']) + \
                np.random.choice(nameparts['3'])
               ).capitalize()
    
    def setLoyaltytoCrown(self):
        if 'ruler' in self.role:
            self.loyalty = 1
    
class nation:
    def __init__(self,name):
        self.name = name
        self.towns = []
        self.diplomacy = {}
        self.ruler = 'none'
        
    def __repr__(self):
        return f"nation of {self.name}"
    
    def addTowns(self,towns):
        self.towns = [town for town in towns 
                         if town.diplomacy.get('nation','none')==self.name]
        
    def getCapitol_str(self):
        c = [t.name for t in self.towns if t.type == 'capitol']
        return c[0]

    def getRuler_str(self):
        return self.ruler.name
    
    def getRuler(self):
        return self.ruler
    
    def addDiplomacy(self,nations):
        otherNations = [n for n in nations.values() if n != self.name]
        for o in otherNations:
            self.diplomacy[o] = {'favor':0,'stance':'neutral'} 
            
    def appointRuler(self):
        t = self.getCapitol_str()
        self.ruler = person(f'ruler of {self.name}',t)
        
    def getRandomCity(self):
        return np.random.choice(self.towns)

    
n = nation('Clearcombe')
n.addTowns(towns)
n.addDiplomacy(landscape['nations'])
n.appointRuler()
print(n.name)
print(n.ruler)
print(n.towns)
print(n.diplomacy)

FileNotFoundError: [Errno 2] File b'namegen.csv' does not exist: b'namegen.csv'

In [ ]:
np.random.choice(n.towns)

In [ ]:
n.ruler.temperment

In [ ]:
nations = []
for x in landscape['nations'].values():
    n = nation(x)
    n.addTowns(towns)
    n.addDiplomacy(landscape['nations'])
    n.appointRuler()
    nations.append(n)

In [ ]:
nations

In [ ]:
nations[0].name,nations[0].diplomacy

In [ ]:
def getNation(a):
    return [n for n in nations if n.name ==a][0]

def alterFavor(a,o,s):
    """
    a = the target nation (obj). a will not change. O's favor of A will change.
    o = the nation(s) who's favor is change. O's favor of a will change
    (a,[o]) each nation in o's favor of a is changed by s
    (a,o) o's favor of a is changed by s
    ([a],o) o's favor of each nation in a is changed by s
    s = amount of change (int)
    """
    if type(o) == list:
        for i in o:
            i.diplomacy[a.name]['favor'] += s
    elif type(a) == list:
         for i in a:
            o.diplomacy[i.name]['favor'] += s       
    else:
        o.diplomacy[a.name]['favor'] += s
    
getNation('Clearward').diplomacy

In [ ]:
alterFavor(getNation('Godtown'),[getNation('Clearward'),getNation('Clearcombe')],-1)    
getNation('Clearward').diplomacy

In [ ]:
nations[1].getCapitol_str()

In [ ]:
events = pd.DataFrame(columns=['n_subjects',
                              'n_objects',
                              'event',
                              'effect_var',
                              'effect'])
events.loc['raiders'] = [1,3,'raiders from {a}, plundered {o}','favor',-1]
events.loc['marrage'] = [1,1,'a princess from {o} was married to {a}','favor',3]
events.loc['temple'] = [1,0,'a great temple was founded in the nation of {a}','buildings','temple']
events

In [ ]:
landscape['eon'] = 10
landscape['chaos'] = .6

In [ ]:
def wheel_of_fate(events,nations,verbose=True):
    """
    a = the subject: the agent, or place where the event happened
    o = the object: the reciever of the event
    """
    choice = np.random.choice(events.index)
    event = events.loc[choice]
    a = np.random.choice(nations,event.n_subjects,replace=False)
    o = np.random.choice(nations,event.n_objects,replace=False)
    if all([len(a)>0,len(o)>0]):
        for ai in a:
            alterFavor(ai,[oi for oi in list(o) if oi!=ai],event.effect)         
    if verbose:
        text = (event.event.replace('{o}',str(o)).replace('{a}',str(a)))    
        print(text)
    
    
#now to run through the eons and let fate happen
for e in range(landscape['eon']):
    if np.random.random_sample()<landscape['chaos']:
        wheel_of_fate(events,nations)
    else:
        print(f'{e}: \'twas a quiet time')

In [ ]:
getNation('Godtown').diplomacy